## KOA Notebook
This notebook is focused on retrieving and spatially visualizing KOA metadata but also includes data from other sources.  First, the KOA observations from all instruments are searched for a cone on the sky centered on rho Ophiuchus (and optionally time range), then a list of protostars in that region is downloaded from CDS Vizier. A mosaic image from Spitzer is retrieved and processed slightly and finally the interactive visualizer showing all this is initialized.

## Setup
Import some packages and define a utility function.

In [3]:
# Allways run this cell.

import os
import urllib.request
from astropy.io import ascii
from MontagePy.main import mShrink


# General function for retrieving URL to file

def retrieve_url_to_file(url, filename):
    
    try:
        urllib.request.urlretrieve(url, filename)
        print(f"Successfully downloaded {url} to {filename}")
    except Exception as e:
        print(f"An error occurred: {e}")

## Parameters and Object Lookup

In [4]:
from astroquery.simbad import Simbad

object    = 'rho Oph'
radius    = 0.5
workspace = ('~/data/' + object).replace(' ', '_')

result = Simbad.query_object(object)

ra  = result[0]['ra']
dec = result[0]['dec']

print('object:    ', object)
print('region:    ', ra, dec, radius)
print('workspace: ', workspace)

object:     rho Oph
region:     246.39632358927884 -23.44717083027709 0.5
workspace:  ~/data/rho_Oph


## KOA Holdings

The web services underpinning the KOA interactive interface are can also be accessed directly via URL.  First we determine which instruments have data for a user-selected region:

In [5]:
# Retrieve metadata from KOA. This requires ra, dec and radius (in degrees)
# and supports a time range (in MJD):

tblfile   = 'counts.tbl'

ftbl = os.path.expanduser(workspace) + '/tbl/' + tblfile

urlbase = 'https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?'
urlbase = urlbase + 'ra=' + str(ra) + '&dec=' + str(dec) 
urlbase = urlbase + '&radius=' + str(radius)
urlbase = urlbase + '&tmin=-1.e9&tmax=1.e9'

retrieve_url_to_file(urlbase, ftbl)

data = ascii.read(ftbl)

print(data)

for row in data:
    catalog = row['identifier'].lower()
    
    tblfile = catalog + '.tbl'

    ftbl = os.path.expanduser(workspace) + '/tbl/' + tblfile 

    url = urlbase + '&catalog=' + catalog

    retrieve_url_to_file(url, ftbl)


Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=0.5&tmin=-1.e9&tmax=1.e9 to /Users/jcg/data/rho_Oph/tbl/counts.tbl
identifier count
---------- -----
    DEIMOS     2
     HIRES    13
       LWS     8
     NIRC2  1740
      NIRC    72
     NIRES    18
   NIRSPEC    61
    OSIRIS    29
    GUIDER   143
Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=0.5&tmin=-1.e9&tmax=1.e9&catalog=deimos to /Users/jcg/data/rho_Oph/tbl/deimos.tbl
Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=0.5&tmin=-1.e9&tmax=1.e9&catalog=hires to /Users/jcg/data/rho_Oph/tbl/hires.tbl
Successfully downloaded https://koa.ipac.caltech.edu/cgi-bin/mSearch/nph-mSearch?ra=246.39632358927884&dec=-23.44717083027709&radius=0.5&tmin=-1.e9&tmax=1.e9&catalog=lws to /Users/jcg/d

## $\rho$ Ophiuchus Protostars

Frequently, it helps to compare KOA search results with catalogs and/or image metadata from other sources. There are thousands of these available via TAP (Table Access Protocol) and SIA (Simple Image Access) services. Here we illustrate this by retrieving a table of protostars from CDS VizieR.  This particular catalog makes sense for the region we have chosen (rho Oph).

In [6]:
from astropy.io import ascii

# Retrieve data from CDS VizieR TAP service:
# "218GHz obs. of embedded protostars in Ophiuchus (Lindberg+, 2017)" [J/ApJ/835/3]

workspace = '~/data/rhoOph'
csvfile   = 'protostars.csv'
tblfile   = 'protostars.tbl'

fcsv = os.path.expanduser(workspace) + '/tbl/' + csvfile
ftbl = os.path.expanduser(workspace) + '/tbl/' + tblfile

url = 'http://tapvizier.u-strasbg.fr/TAPVizieR/tap/sync?'
url = url + 'request=doquery&lang=adql&query='
url = url + 'select+*+from+"J/ApJ/835/3/table1"'
url = url + "&format=csv"

retrieve_url_to_file(url, fcsv)


# Convert VizieR CSV format to IPAC table

data = ascii.read(fcsv)
data.rename_column('RAJ2000', 'ra')
data.rename_column('DEJ2000', 'dec')
data.write(ftbl, format="ipac", overwrite=True)

print('Converted to', ftbl)

Successfully downloaded http://tapvizier.u-strasbg.fr/TAPVizieR/tap/sync?request=doquery&lang=adql&query=select+*+from+"J/ApJ/835/3/table1"&format=csv to /Users/jcg/data/rhoOph/tbl/protostars.csv
Converted to /Users/jcg/data/rhoOph/tbl/protostars.tbl


## Download Background Image

For the region visualizer below we need some sort of FITS image.  If you have your own or if there is one available on the network, we can download that.  There are a few survey datasets from which we can generate an image (<i>e.g.</i>, 2MASS) though perhaps not at too high a resolution.  As a fallback, we can generate a blank image with whatever WCS we choose, basically resulting in a map of the observations with no underlying image.  

For this example, we are going to use a mosaic made from Spitzer image data for the rho Oph region.  This takes a little time to download, so be patient:

In [7]:
file = 'rhoOph_large.fits'

fimg = os.path.expanduser(workspace) + '/fits/' + file

url = 'https://irsa.ipac.caltech.edu/data/SPITZER/Enhanced/SEIP/images/'
url = url + '4/0015/40015561/7/40015561-87/40015561.40015561-87.IRAC.1.mosaic.fits'

retrieve_url_to_file(url, fimg)

Successfully downloaded https://irsa.ipac.caltech.edu/data/SPITZER/Enhanced/SEIP/images/4/0015/40015561/7/40015561-87/40015561.40015561-87.IRAC.1.mosaic.fits to /Users/jcg/data/rhoOph/fits/rhoOph_large.fits


### Shrink the Image

Depending on your image and application, it may make sense to resize the FITS image.  In our case the image is 4000x4000 pixels, so shrinking it in both dimensions by a factor of four will speed up the redraws.

In [8]:
infile        = 'rhoOph_large.fits'
outfile       = 'rhoOph.fits'
shrink_factor = 4

fin  = os.path.expanduser(workspace) + '/fits/' + infile
fout = os.path.expanduser(workspace) + '/fits/' + outfile

mShrink(fin, fout, shrink_factor)

{'status': '0', 'time': 0.0}

### Defining the Layout of the Display

Unless we tell it, there is no way for the viewer to decide what data from the workspace to display, how to stretch the image (or images in the case of 3-color data), what symbols to use for point source catalogs (and their scaling an color), whether to show coordinate grids, and so on.

Some displays can be handled with defaults but this one is a little too complicated for that to work well.  So we will go the other route here of defining everything ourselves through a JSON layout file. For more information on mViewer, see http://montage.ipac.caltech.edu/mViewer.

In [9]:
import json

layout = {
  "Image Label": "rho Oph: Spitzer IRAC",

  "Grayscale":
  [
    {
      "File": "fits/IRAC1.fits",
      "Color Table": "Grayscale",
      "Stretch Mode": "Gaussian-log",
      "Min": "-0.5", "Min Units": "sigma",
      "Max": "max", "Max Units": "sigma"
    }
  ],

  "Overlays":
  [
    {"Type": "Catalog", "Color": "ffffff", "File": "tbl/deimos.tbl",     "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ff8080", "File": "tbl/hires.tbl",      "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "8080ff", "File": "tbl/lris.tbl",       "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffff80", "File": "tbl/lws.tbl",        "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "80ffff", "File": "tbl/nirc.tbl",       "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ccccff", "File": "tbl/nirc2.tbl",      "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffbf00", "File": "tbl/osiris.tbl",     "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "80ff80", "File": "tbl/esi.tbl",        "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ff80ff", "File": "tbl/nires.tbl",      "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffaaaa", "File": "tbl/nirspec.tbl",    "Size of Ref Value":  3.0, "Symbol": "box"},
    {"Type": "Catalog", "Color": "ffffff", "File": "tbl/protostars.tbl", "Size of Ref Value":  5.0, "Symbol": "circle"},
    {"Type": "Eq grid", "Color": "808000"}
  ]
}

filename = os.path.expanduser(workspace) + "/layout.json"

with open(filename, 'w') as file:
    json.dump(layout, file, indent=2)

print("JSON layout written to", filename)

JSON layout written to /Users/jcg/data/rhoOph/layout.json


## Region Vizualization

Please read this description before running the next cell.  The 'koaViewer' app helps you visualize the data you have downloaded into the workspace.  This workspace contains a 'tbl' subdirectory with all the table files you have downloaded (both source tables and image metadata tables) and a 'fits' directory with a FITS image file or files.  The data can easily get too involved for a single simple display (though what we have done through this Notebook is simple enough), but even here someone has to specify things like which tables to actually display, the colors of overlays in the image display, symbol sizes, <i>etc.</i>

So the actual image display gets defined by a JSON structure file ('layout.json') in the top level of the workspace.  

There are three ways to build this JSON.  If no layout.json exists when koaViewer starts up, it will analyze the contents of the workspace and take a guess at the structure using a few simple rules.  This is likely to be sub-optimal.  Alternatively, knowing what we have done in the Notebook to construct the workspace, we can construct the JSON manually (<i>i.e.,</i> setting up the JSON in a Notebook cell).  Finally, once we have a starting point, the koaViewer application has layout editing pop-ups where we can fine tune the details (like color and column-base symbol sizes).

Any changes to the this Notebook (such as the location/search radius in the first step) can affect how many tables there are to display, hard-coded JSON is probably inadequate, so we will go with first option and let koaViewer define the JSON we start with.

To start koaViewer, run the next cell and activate the link it produces.

In [11]:
from koaviewer import koaViewer

viewer = koaViewer()

viewer.run(workspace)

Dash app running on http://127.0.0.1:62625/
